# Building an End-to-End System: A Customer Service Chatbot

This notebook is the grand finale! We're assembling all the individual components -- input moderation, prompt chaining, and output evaluation - into a single, fully functional, end-to-end customer service chatbot.

---
### The Complete AI Pipeline
Every message a user sends now goes through this robust, multi-step pipeline to ensure the response is safe, relevant, and factually correct:

1.  **Input Moderation**: First, the user's query is checked for safety.
2.  **Product Extraction**: An LLM call parses the query and extracts product/category names into a structured JSON list.
3.  **Data Retrieval**: The system looks up detailed information for the extracted items from a local product catalog.
4.  **Response Generation**: A second LLM synthesizes the retrieved data into a helpful, conversational answer.
5.  **Output Moderation**: The AI's generated response is checked for safety.
6.  **Fact-Checking**: A final "evaluator" LLM call confirms the response is factually grounded in the source data.
7.  **Final Decision**: If the response passes all checks, it's sent to the user. Otherwise, a safe fallback message is used.

---
### New Features in this Notebook
* **Conversation Memory**: The system now uses a `context` variable to store the chat history, allowing for multi-turn, stateful conversations.
* **Interactive Chat UI**: We use the `panel` library to create a simple graphical interface, turning our Python script into an interactive application.

In [1]:
%run pipeline.py

In [2]:
user_input = "Tell me about the smartx pro phone and the fotosnap camera, the dslr one. \
Also tell me about your tvs"

response, _ = run_pipeline(user_input, [])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.


The **SmartX ProPhone** is a fantastic choice if you're looking for a powerful smartphone with advanced features. It boasts a 6.1-inch display, 128GB of storage, and a 12MP dual camera, all wrapped up with 5G connectivity. This phone is perfect for anyone who loves capturing high-quality photos and videos on the go. Priced at $899.99, it offers a great balance of performance and value.

For photography enthusiasts, the **FotoSnap DSLR Camera** is a versatile option. With a 24.2MP sensor, 1080p video capability, and a 3-inch LCD, this camera is designed to capture stunning photos and videos. It also features interchangeable lenses, allowing you to adapt to different shooting conditions. At $599.99, it's a great in

### Function that collects user and assistant messages over time

In [3]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    
    response, context = run_pipeline(user_input, context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:    ', pn.pane.Markdown(user_input, width=600, style={'background-color': '#DCF8C6'})))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!

In [4]:
panels = []

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, height=300, loading_indicator=True)

In [13]:
import json

def print_conversation(context):
    """
    Prints the conversation history in a readable, formatted way.
    """
    if not context:
        print("Context is empty.")
        return

    print("="*25 + " CONVERSATION LOG " + "="*25)
    for message in context:
        role = message.get('role', 'N/A').upper()
        content = message.get('content', '')
        
        print(f"\n----- ROLE: {role} -----")
        print(content)
            
    print("\n" + "="*22 + " END OF CONVERSATION " + "="*22)

In [14]:
print_conversation(context)

========================= CONVERSATION LOG =========================

----- ROLE: SYSTEM -----
You are Service Assistant

----- ROLE: USER -----
```Tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also tell me about your tvs.```

----- ROLE: ASSISTANT -----
Relevant product information:
{
    "name": "SmartX ProPhone",
    "category": "Smartphones and Accessories",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "A powerful smartphone with advanced camera features.",
    "price": 899.99
}
{
    "name": "FotoSnap DSLR Camera",
    "category": "Cameras and Camcorders",
    "brand": "FotoSnap",
    "model_number": "FS-DSLR200",
    "warranty": "1 year",
    "rating": 4.7,
    "features": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchange